In [ ]:
# !pip install transformers torch matplotlib seaborn

In [5]:
import os
import copy
import torch
from transformers import GPT2Model, GPT2Tokenizer
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Define the list of model names
model_names = [
    'shng2025/GPT-Valkyrie_RMSN-124m__noNorm__SQuAD',
    'shng2025/GPT-Valkyrie_RMSN-124m__AttnOnly__SQuAD',
    'shng2025/GPT-Valkyrie_RMSN-124m__FFNonly__SQuAD',
    'shng2025/GPT-Valkyrie_RMSN-124m__baseModel__SQuAD'
]

# Sample input text
sample_text = "Once upon a time in a land far, far away, there lived a wise old owl."

# Directory to save attention plots
output_dir = "attention_plots"
os.makedirs(output_dir, exist_ok=True)

# Initialize the tokenizer once since all models share the same tokenizer settings
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.cls_token = "[CLS]"
tokenizer.add_special_tokens({'cls_token': '[CLS]'})  # Ensure special tokens are added


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1

In [ ]:
# Function to plot a single attention heatmap
def plot_attention(ax, attention, layer, head, model_name):
    sns.heatmap(attention, cmap='viridis', ax=ax, cbar=False)
    ax.set_title(f'{model_name}\nLayer {layer+1} Head {head+1}', fontsize=8)
    ax.set_xlabel('Key Positions', fontsize=6)
    ax.set_ylabel('Query Positions', fontsize=6)
    ax.tick_params(labelsize=6)

# Initialize the figure
num_models = len(model_names)
num_layers = 12  # Assuming GPT-2 base with 12 layers
num_heads = 12   # Assuming GPT-2 base with 12 heads per layer

# Define subplot grid: models vertically, layers within models, heads within layers
fig_height = num_models * num_layers * 0.5  # Adjust the height per layer
fig_width = num_heads * 1.0  # Adjust the width per head
fig, axes = plt.subplots(num_models * num_layers, num_heads, figsize=(num_heads * 1.0, num_models * num_layers * 0.5))

# Ensure axes is a 2D array even if num_heads=1
if num_models * num_layers == 1:
    axes = np.array([axes])
elif num_heads == 1:
    axes = axes.reshape(num_models * num_layers, 1)

# Iterate over each model
for model_idx, model_name in enumerate(model_names):
    print(f"Processing model: {model_name}")

    # Load the model with attention outputs
    model = GPT2Model.from_pretrained(model_name, output_attentions=True)
    model.eval()  # Set model to evaluation mode

    # Tokenize the input
    inputs = tokenizer(sample_text, return_tensors='pt')
    input_ids = inputs['input_ids']

    # Get the attention outputs
    with torch.no_grad():
        outputs = model(**inputs)
    attentions = outputs.attentions  # List of tensors: one for each layer

    # Iterate over each layer
    for layer in range(num_layers):
        # Iterate over each head in the layer
        for head in range(num_heads):
            # Calculate the absolute position in the axes grid
            plot_idx = model_idx * num_layers + layer
            ax = axes[plot_idx, head] if num_heads > 1 else axes[plot_idx]

            # Extract attention weights for this layer and head
            # Shape: (batch_size, num_heads, seq_length, seq_length)
            attention = attentions[layer][0, head].cpu().numpy()

            # Plot the attention heatmap
            plot_attention(ax, attention, layer, head, model_name.split('/')[-1])

        if (layer + 1) % 4 == 0:
            print(f"  Processed Layer {layer+1}/{num_layers}")

# Adjust layout
plt.tight_layout()

# Add a main title
plt.suptitle('Attention Head Comparisons Across Models', fontsize=16, y=1.02)

# Save the figure
fig_file = os.path.join(output_dir, 'all_models_attention_comparison.png')
plt.savefig(fig_file, bbox_inches='tight', dpi=300)
print(f"All attention heatmaps have been plotted and saved to {fig_file}")

# Show the plot
plt.show()

Processing model: shng2025/GPT-Valkyrie_RMSN-124m__noNorm__SQuAD
  Processed Layer 4/12


# Averaging Heads across each Layer

In [ ]:
import os
import torch
from transformers import GPT2Model, GPT2Tokenizer
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib.gridspec import GridSpec

# Define the list of model names
model_names = [
    'shng2025/GPT-Valkyrie_RMSN-124m__noNorm__SQuAD',
    'shng2025/GPT-Valkyrie_RMSN-124m__AttnOnly__SQuAD',
    'shng2025/GPT-Valkyrie_RMSN-124m__FFNonly__SQuAD',
    'shng2025/GPT-Valkyrie_RMSN-124m__baseModel__SQuAD'
]

# Sample input text
sample_text = "Once upon a time in a land far, far away, there lived a wise old owl."

# Directory to save attention plots (optional)
output_dir = "attention_plots_layer_avg"
os.makedirs(output_dir, exist_ok=True)

# Initialize the tokenizer once since all models share the same tokenizer settings
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.cls_token = "[CLS]"
# Add special tokens only once
if '[CLS]' not in tokenizer.get_vocab():
    tokenizer.add_special_tokens({'cls_token': '[CLS]'})

# Function to plot a single attention heatmap
def plot_attention(ax, attention, layer, model_name):
    sns.heatmap(attention, cmap='viridis', ax=ax, cbar=False)
    ax.set_title(f'{model_name}\nLayer {layer+1}', fontsize=8)
    ax.set_xlabel('Key Positions', fontsize=6)
    ax.set_ylabel('Query Positions', fontsize=6)
    ax.tick_params(labelsize=6)

# Iterate over each model
for model_name in model_names:
    print(f"Processing model: {model_name}")

    # Load the model with attention outputs
    model = GPT2Model.from_pretrained(model_name, output_attentions=True)
    model.eval()  # Set model to evaluation mode

    # Tokenize the input
    inputs = tokenizer(sample_text, return_tensors='pt')
    input_ids = inputs['input_ids']

    # Get the attention outputs
    with torch.no_grad():
        outputs = model(**inputs)
    attentions = outputs.attentions  # List of tensors: one for each layer

    num_layers = len(attentions)
    num_heads = attentions[0].shape[1]  # Assuming all layers have the same number of heads
    seq_length = attentions[0].shape[-1]

    print(f"Number of layers: {num_layers}, Number of heads per layer: {num_heads}, Sequence length: {seq_length}")

    # Prepare a grid for plotting: e.g., 4 rows (layers) x 3 cols (heads) for 12 layers
    # Adjust grid size based on number of layers
    cols = 6  # Number of columns in the grid
    rows = int(np.ceil(num_layers / cols))

    fig, axs = plt.subplots(rows, cols, figsize=(cols * 2, rows * 2))
    fig.suptitle(f'Layer-wise Averaged Attention - {model_name.split("/")[-1]}', fontsize=12)

    for layer in range(num_layers):
        # Average attention across heads
        attention_avg = attentions[layer].mean(dim=1).squeeze(0).cpu().numpy()

        # Determine subplot position
        row = layer // cols
        col = layer % cols

        ax = axs[row, col] if rows > 1 else axs[col]

        # Plot the averaged attention
        plot_attention(ax, attention_avg, layer, model_name.split('/')[-1])

    # Remove any empty subplots
    total_subplots = rows * cols
    if num_layers < total_subplots:
        for empty in range(num_layers, total_subplots):
            row = empty // cols
            col = empty % cols
            ax = axs[row, col] if rows > 1 else axs[col]
            ax.axis('off')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    # Save the figure
    save_path = os.path.join(output_dir, f'{model_name.split("/")[-1]}_layer_avg_attention.png')
    plt.savefig(save_path, dpi=300)
    plt.close()
    print(f"Saved averaged attention plots for model: {model_name}\n")

print("All models have been processed and layer-averaged attention heatmaps have been saved.")

# Averaging Heads across each Layer AND THEN Averaging each Layer

In [ ]:
import os
import torch
from transformers import GPT2Model, GPT2Tokenizer
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Define the list of model names
model_names = [
    'shng2025/GPT-Valkyrie_RMSN-124m__noNorm__SQuAD',
    'shng2025/GPT-Valkyrie_RMSN-124m__AttnOnly__SQuAD',
    'shng2025/GPT-Valkyrie_RMSN-124m__FFNonly__SQuAD',
    'shng2025/GPT-Valkyrie_RMSN-124m__baseModel__SQuAD'
]

# Sample input text
sample_text = "Once upon a time in a land far, far away, there lived a wise old owl."

# Directory to save overall attention plots
output_dir = "attention_plots_overall_avg"
os.makedirs(output_dir, exist_ok=True)

# Initialize the tokenizer once since all models share the same tokenizer settings
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.cls_token = "[CLS]"
# Add special tokens only once
if '[CLS]' not in tokenizer.get_vocab():
    tokenizer.add_special_tokens({'cls_token': '[CLS]'})

# Function to plot a single overall attention heatmap
def plot_overall_attention(ax, attention, model_name):
    sns.heatmap(attention, cmap='viridis', ax=ax, cbar=True)
    ax.set_title(f'Overall Averaged Attention\n{model_name}', fontsize=10)
    ax.set_xlabel('Key Positions', fontsize=8)
    ax.set_ylabel('Query Positions', fontsize=8)
    ax.tick_params(labelsize=6)

# Initialize a single figure with subplots for all models
fig, axs = plt.subplots(2, 2, figsize=(10, 8))  # 2x2 grid for four models
fig.suptitle('Overall Averaged Attention Across Models', fontsize=16)

# Iterate over each model
for idx, model_name in enumerate(model_names):
    print(f"Processing model: {model_name}")

    # Load the model with attention outputs
    model = GPT2Model.from_pretrained(model_name, output_attentions=True)
    model.eval()  # Set model to evaluation mode

    # Tokenize the input
    inputs = tokenizer(sample_text, return_tensors='pt')
    input_ids = inputs['input_ids']

    # Get the attention outputs
    with torch.no_grad():
        outputs = model(**inputs)
    attentions = outputs.attentions  # List of tensors: one for each layer

    num_layers = len(attentions)
    num_heads = attentions[0].shape[1]  # Assuming all layers have the same number of heads
    seq_length = attentions[0].shape[-1]

    print(f"Number of layers: {num_layers}, Number of heads per layer: {num_heads}, Sequence length: {seq_length}")

    # Average attention across heads for each layer
    layer_avg_attentions = [attn.mean(dim=1).squeeze(0).cpu().numpy() for attn in attentions]

    # Further average across layers
    overall_avg_attention = np.mean(layer_avg_attentions, axis=0)

    # Determine subplot position
    row = idx // 2
    col = idx % 2
    ax = axs[row, col]

    # Plot the overall averaged attention
    plot_overall_attention(ax, overall_avg_attention, model_name.split('/')[-1])

    print(f"Completed processing for model: {model_name}\n")

# Adjust layout
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

# Save the figure
save_path = os.path.join(output_dir, 'overall_averaged_attention.png')
plt.savefig(save_path, dpi=300)
plt.close()
print(f"Overall averaged attention heatmaps have been saved to {save_path}")

# Display the plot (optional)
plt.show()


# https://chatgpt.com/c/66effe31-bd64-8003-baf2-f8977d9a3a10?model=o1-mini

Your approach to analyzing ablated transformer models by examining gradient magnitudes is insightful and provides valuable information about the optimization landscape of each model. To further enrich your analysis and gain a more comprehensive understanding of how different normalization strategies impact transformer architectures, you can incorporate several additional methods. Below are some recommended techniques, along with explanations and implementation guidance, tailored to fit within the scope of your IB Extended Essay.

---

## **1. Attention Head Analysis**

### **a. Description**
Transformer models rely heavily on attention mechanisms. Each layer contains multiple attention heads that focus on different parts of the input. Analyzing these attention patterns can reveal how normalization strategies influence the model's focus and information flow.

### **b. Why It’s Useful**
- **Understanding Focus:** Determines whether certain heads become more specialized or diffuse based on normalization.
- **Identifying Redundancy:** Helps identify if some heads become redundant or overly concentrated.

### **c. How to Implement**
1. **Extract Attention Weights:**
   During inference, extract the attention weights from each head in each layer.
   ```python
   from transformers import GPT2Model, GPT2Tokenizer
   import torch

   tokenizer = GPT2Tokenizer.from_pretrained('your_model')
   model = GPT2Model.from_pretrained('your_model', output_attentions=True)
   inputs = tokenizer("Your input text here", return_tensors='pt')
   outputs = model(**inputs)
   attentions = outputs.attentions  # List of attention weights for each layer
   ```

2. **Visualize Attention Maps:**
   Use visualization libraries like Matplotlib or Seaborn to plot attention distributions.
   ```python
   import matplotlib.pyplot as plt
   import seaborn as sns

   def plot_attention(attention, layer, head):
       sns.heatmap(attention[layer][0][head].detach().numpy(), cmap='viridis')
       plt.title(f'Layer {layer+1} Head {head+1} Attention')
       plt.xlabel('Key Positions')
       plt.ylabel('Query Positions')
       plt.show()

   # Example: Plotting attention for layer 0, head 0
   plot_attention(attentions, layer=0, head=0)
   ```

3. **Analyze Patterns:**
   - **Focused vs. Diffuse Attention:** Determine if normalization leads to more focused attention (peaky distributions) or diffuse attention (more spread out).
   - **Cross-Head Specialization:** Check if different heads specialize in different types of attention (e.g., syntactic vs. semantic).

### **d. Insights Gained**
- **Normalization Impact:** How normalization affects the distribution and focus of attention across heads.
- **Model Efficiency:** Whether normalization leads to more efficient use of attention heads.

---

## **2. Activation Distribution Comparison**

### **a. Description**
Analyzing the distribution of activations across different layers and components can provide insights into how normalization strategies influence the internal representations of the model.

### **b. Why It’s Useful**
- **Distribution Shifts:** Identifies how normalization affects the scale and spread of activations.
- **Dynamic Range:** Assesses whether normalization leads to more stable activation ranges across layers.

### **c. How to Implement**
1. **Extract Activations:**
   Modify the model to return activations from each layer.
   ```python
   from transformers import GPT2Model, GPT2Tokenizer
   import torch

   tokenizer = GPT2Tokenizer.from_pretrained('your_model')
   model = GPT2Model.from_pretrained('your_model', output_hidden_states=True)
   inputs = tokenizer("Your input text here", return_tensors='pt')
   outputs = model(**inputs)
   hidden_states = outputs.hidden_states  # List of hidden states for each layer
   ```

2. **Compute Statistics:**
   Calculate mean, variance, skewness, and kurtosis for activations in each layer.
   ```python
   import numpy as np

   def compute_activation_stats(hidden_states):
       stats = []
       for layer in hidden_states:
           layer_np = layer.detach().numpy()
           mean = np.mean(layer_np)
           var = np.var(layer_np)
           skew = np.mean((layer_np - mean)**3) / (np.var(layer_np)**1.5)
           kurt = np.mean((layer_np - mean)**4) / (np.var(layer_np)**2)
           stats.append({'mean': mean, 'variance': var, 'skewness': skew, 'kurtosis': kurt})
       return stats

   activation_stats = compute_activation_stats(hidden_states)
   ```

3. **Visualize Distributions:**
   Use box plots or histograms to compare statistics across models.
   ```python
   import matplotlib.pyplot as plt

   def plot_activation_stats(stats, metric):
       values = [layer[metric] for layer in stats]
       plt.plot(values, label=f'{metric.capitalize()}')
       plt.xlabel('Layer')
       plt.ylabel(metric.capitalize())
       plt.title(f'Activation {metric.capitalize()} Across Layers')
       plt.legend()
       plt.show()

   # Example: Plotting mean activations
   plot_activation_stats(activation_stats, 'mean')
   ```

### **d. Insights Gained**
- **Stability:** Whether normalization leads to more stable activations across layers.
- **Scaling Effects:** How normalization affects the scale of representations, potentially impacting learning dynamics.

---

## **3. Representational Similarity Analysis (RSA) or Centered Kernel Alignment (CKA)**

### **a. Description**
RSA and CKA are techniques used to compare the internal representations of different models or different layers within a model. They measure the similarity between activation patterns, providing a quantitative way to assess how different normalization strategies affect learned representations.

### **b. Why It’s Useful**
- **Comparative Insights:** Quantifies how similar or different the representations are across models.
- **Layer-Wise Analysis:** Allows comparison of specific layers, identifying where normalization has the most impact.

### **c. How to Implement**
1. **Extract Hidden States:**
   Similar to the activation extraction step above.

2. **Compute CKA:**
   Use libraries like `keras-rl` or implement CKA from scratch. Here’s a simplified example using CKA:
   ```python
   import numpy as np
   from sklearn.metrics.pairwise import linear_kernel

   def center_matrix(X):
       return X - X.mean(axis=0)

   def linear_cka(X, Y):
       X_centered = center_matrix(X)
       Y_centered = center_matrix(Y)
       K = linear_kernel(X_centered, X_centered)
       L = linear_kernel(Y_centered, Y_centered)
       return np.sum(K * L) / (np.sqrt(np.sum(K * K)) * np.sqrt(np.sum(L * L)))

   # Example: Comparing layer 0 of two models
   cka_score = linear_cka(hidden_states_model1[0].reshape(-1, hidden_states_model1[0].shape[-1]),
                          hidden_states_model2[0].reshape(-1, hidden_states_model2[0].shape[-1]))
   print(f'CKA Score for Layer 0: {cka_score}')
   ```

3. **Interpret Results:**
   - **High Similarity:** Indicates that the normalization strategies lead to similar representations.
   - **Low Similarity:** Suggests that normalization strategies result in different internal representations.

### **d. Insights Gained**
- **Representation Divergence:** How different normalization approaches cause models to learn distinct representations.
- **Layer-Specific Effects:** Identifies which layers are most affected by normalization strategies.

---

## **4. Weight Distribution and Norm Comparison**

### **a. Description**
Analyzing the distribution and norms of model weights can provide insights into how normalization affects parameter scaling and distribution, potentially impacting learning dynamics and generalization.

### **b. Why It’s Useful**
- **Weight Scaling:** Normalization can influence the scale of weights, affecting training stability and convergence.
- **Distribution Shifts:** Changes in weight distributions can indicate how normalization strategies guide the optimization process.

### **c. How to Implement**
1. **Extract Model Weights:**
   ```python
   model = GPT2Model.from_pretrained('your_model')
   weights = [param.detach().numpy() for param in model.parameters()]
   ```

2. **Compute Statistics:**
   Calculate mean, variance, skewness, and kurtosis for weights in each layer.
   ```python
   def compute_weight_stats(weights):
       stats = []
       for layer_weights in weights:
           mean = np.mean(layer_weights)
           var = np.var(layer_weights)
           skew = np.mean((layer_weights - mean)**3) / (np.var(layer_weights)**1.5)
           kurt = np.mean((layer_weights - mean)**4) / (np.var(layer_weights)**2)
           stats.append({'mean': mean, 'variance': var, 'skewness': skew, 'kurtosis': kurt})
       return stats

   weight_stats = compute_weight_stats(weights)
   ```

3. **Compare Norms:**
   Calculate norms (e.g., L2 norm) of weights across layers and models.
   ```python
   def compute_weight_norms(weights):
       norms = []
       for layer_weights in weights:
           norm = np.linalg.norm(layer_weights)
           norms.append(norm)
       return norms

   weight_norms = compute_weight_norms(weights)
   ```

4. **Visualize and Compare:**
   Use line plots or box plots to compare weight statistics across models.
   ```python
   import matplotlib.pyplot as plt

   def plot_weight_stats(stats, metric, model_name):
       values = [layer[metric] for layer in stats]
       plt.plot(values, label=model_name)
       plt.xlabel('Layer')
       plt.ylabel(metric.capitalize())
       plt.title(f'Weight {metric.capitalize()} Across Layers')
       plt.legend()
       plt.show()

   # Example: Plotting weight variances
   plot_weight_stats(weight_stats_model1, 'variance', 'Model 1')
   plot_weight_stats(weight_stats_model2, 'variance', 'Model 2')
   ```

### **d. Insights Gained**
- **Normalization Effects:** Understanding how different normalization strategies impact weight scaling and distribution.
- **Training Dynamics:** Insights into how normalization influences the optimization landscape through weight behavior.

---

## **5. Probing Tasks**

### **a. Description**
Probing tasks involve training simple classifiers on the model’s internal representations to assess what linguistic or semantic information is encoded at different layers. This method helps in understanding the qualitative aspects of what the model learns.

### **b. Why It’s Useful**
- **Semantic Understanding:** Determines how different normalization strategies affect the encoding of semantic and syntactic information.
- **Layer-Specific Insights:** Identifies at which layers specific types of information are most strongly represented.

### **c. How to Implement**
1. **Select Probing Tasks:**
   Choose tasks like part-of-speech tagging, syntactic parsing, or semantic role labeling.

2. **Extract Representations:**
   Obtain hidden states from each layer for a set of labeled data.
   ```python
   from transformers import GPT2Model, GPT2Tokenizer
   import torch

   tokenizer = GPT2Tokenizer.from_pretrained('your_model')
   model = GPT2Model.from_pretrained('your_model', output_hidden_states=True)
   inputs = tokenizer("Your input text here", return_tensors='pt')
   outputs = model(**inputs)
   hidden_states = outputs.hidden_states  # List of hidden states for each layer
   ```

3. **Train Probing Classifiers:**
   For each layer, train a simple classifier (e.g., logistic regression) to predict the task labels.
   ```python
   from sklearn.linear_model import LogisticRegression
   from sklearn.metrics import accuracy_score

   # Example: Probing for Part-of-Speech Tagging
   def probe_layer(hidden_state, labels):
       # Flatten the hidden state
       X = hidden_state.detach().numpy().reshape(-1, hidden_state.shape[-1])
       y = labels.flatten()
       # Train a logistic regression classifier
       clf = LogisticRegression(max_iter=1000)
       clf.fit(X, y)
       y_pred = clf.predict(X)
       return accuracy_score(y, y_pred)

   # Assume `labels` is a numpy array of POS tags corresponding to the input
   pos_accuracy = probe_layer(hidden_states[0], labels)
   print(f'POS Tagging Accuracy for Layer 0: {pos_accuracy}')
   ```

4. **Compare Across Models:**
   Evaluate and compare probing classifier performance across different models and layers.

### **d. Insights Gained**
- **Information Encoding:** How normalization strategies influence the encoding of different types of linguistic information.
- **Layer Specialization:** Identifies which layers specialize in encoding specific information based on normalization.

---

## **6. Weight Distribution and Norm Comparison**

### **a. Description**
Examining the distribution and norms of model weights can reveal how normalization impacts parameter scaling and distribution, which in turn affects training dynamics and generalization.

### **b. Why It’s Useful**
- **Parameter Scaling:** Normalization can influence the scale of weights, affecting stability and convergence.
- **Distribution Shifts:** Changes in weight distributions can indicate how normalization strategies guide the optimization process.

### **c. How to Implement**
1. **Extract Model Weights:**
   ```python
   model = GPT2Model.from_pretrained('your_model')
   weights = [param.detach().numpy() for param in model.parameters()]
   ```

2. **Compute Statistics:**
   Calculate mean, variance, skewness, and kurtosis for weights in each layer.
   ```python
   def compute_weight_stats(weights):
       stats = []
       for layer_weights in weights:
           mean = np.mean(layer_weights)
           var = np.var(layer_weights)
           skew = np.mean((layer_weights - mean)**3) / (np.var(layer_weights)**1.5)
           kurt = np.mean((layer_weights - mean)**4) / (np.var(layer_weights)**2)
           stats.append({'mean': mean, 'variance': var, 'skewness': skew, 'kurtosis': kurt})
       return stats

   weight_stats = compute_weight_stats(weights)
   ```

3. **Compare Norms:**
   Calculate norms (e.g., L2 norm) of weights across layers and models.
   ```python
   def compute_weight_norms(weights):
       norms = []
       for layer_weights in weights:
           norm = np.linalg.norm(layer_weights)
           norms.append(norm)
       return norms

   weight_norms = compute_weight_norms(weights)
   ```

4. **Visualize and Compare:**
   Use line plots or box plots to compare weight statistics across models.
   ```python
   import matplotlib.pyplot as plt

   def plot_weight_stats(stats, metric, model_name):
       values = [layer[metric] for layer in stats]
       plt.plot(values, label=model_name)
       plt.xlabel('Layer')
       plt.ylabel(metric.capitalize())
       plt.title(f'Weight {metric.capitalize()} Across Layers')
       plt.legend()
       plt.show()

   # Example: Plotting weight variances
   plot_weight_stats(weight_stats_model1, 'variance', 'Model 1')
   plot_weight_stats(weight_stats_model2, 'variance', 'Model 2')
   ```

### **d. Insights Gained**
- **Normalization Effects:** Understanding how different normalization strategies impact weight scaling and distribution.
- **Training Dynamics:** Insights into how normalization influences the optimization landscape through weight behavior.

---

## **7. Loss Landscape Analysis**

### **a. Description**
Loss landscape analysis involves visualizing how the loss function behaves around the current parameters. This can help understand the optimization landscape shaped by different normalization strategies.

### **b. Why It’s Useful**
- **Optimization Insights:** Reveals whether normalization leads to smoother or more rugged loss landscapes.
- **Convergence Behavior:** Helps understand the ease with which models can reach lower loss regions.

### **c. How to Implement**
1. **Generate Perturbations:**
   Apply small perturbations to the model’s parameters in various directions.

2. **Compute Loss:**
   Evaluate the loss for each perturbed parameter set.
   ```python
   import torch
   import numpy as np
   import matplotlib.pyplot as plt

   def plot_loss_landscape(model, input_ids, target_ids, perturbation=0.1):
       # Define perturbation directions
       directions = [torch.randn_like(param) for param in model.parameters()]
       losses = []
       for direction in directions:
           perturbed_model = copy.deepcopy(model)
           for param, dir in zip(perturbed_model.parameters(), directions):
               param.data += perturbation * dir
           outputs = perturbed_model(input_ids)
           loss = compute_loss(outputs, target_ids)
           losses.append(loss.item())
       # Plot the loss landscape
       plt.scatter(range(len(losses)), losses)
       plt.xlabel('Perturbation Direction')
       plt.ylabel('Loss')
       plt.title('Loss Landscape')
       plt.show()

   # Example usage
   plot_loss_landscape(model, input_ids, target_ids)
   ```

3. **Visualize Loss Surface:**
   Create 2D or 3D plots to visualize how loss varies with parameter perturbations.

### **d. Insights Gained**
- **Smoothness:** Smoother loss landscapes suggest better optimization properties influenced by normalization.
- **Local Minima:** Understanding the proximity to local minima and how normalization affects their accessibility.

---

## **8. Robustness and Stability Analysis**

### **a. Description**
Assessing how robust and stable each model is under various input perturbations or noisy conditions can provide insights into the effects of normalization strategies on model generalization and resilience.

### **b. Why It’s Useful**
- **Generalization:** Robust models are better at handling unseen or noisy data.
- **Stability:** Stable training dynamics indicate effective normalization.

### **c. How to Implement**
1. **Input Perturbations:**
   Apply noise, adversarial attacks, or other perturbations to input data.

2. **Evaluate Performance:**
   Measure how performance metrics degrade under perturbations.
   ```python
   from transformers import GPT2Tokenizer, GPT2Model
   import torch

   tokenizer = GPT2Tokenizer.from_pretrained('your_model')
   model = GPT2Model.from_pretrained('your_model')

   def add_noise(text, noise_level=0.1):
       words = text.split()
       num_noisy = int(len(words) * noise_level)
       noisy_indices = np.random.choice(len(words), num_noisy, replace=False)
       for idx in noisy_indices:
           words[idx] = ''.join(np.random.choice(list('abcdefghijklmnopqrstuvwxyz'), size=3))
       return ' '.join(words)

   original_text = "This is a sample input for robustness testing."
   noisy_text = add_noise(original_text, noise_level=0.2)

   inputs_original = tokenizer(original_text, return_tensors='pt')
   inputs_noisy = tokenizer(noisy_text, return_tensors='pt')

   outputs_original = model(**inputs_original)
   outputs_noisy = model(**inputs_noisy)

   # Compare outputs or downstream task performance
   ```

3. **Measure Performance Degradation:**
   Compare metrics like accuracy, F1 score, or perplexity between clean and perturbed inputs.

### **d. Insights Gained**
- **Resilience:** How normalization affects the model’s ability to handle noisy or adversarial inputs.
- **Generalization:** Indicates the robustness of learned representations under different conditions.

---

## **9. Summary of Recommended Methods**

Given the constraints of your **IB Extended Essay** (4,000 words) and the focus on **Text Generation**, **QA**, and **Summarization**, it's essential to prioritize methods that offer significant insights without overcomplicating your analysis. Here’s a summary of recommended methods to complement your gradient visualization:

1. **Attention Head Analysis:**
   - **Why:** Reveals how normalization impacts the focus and specialization of attention heads.
   - **Implementation:** Extract and visualize attention weights; analyze patterns.

2. **Activation Distribution Comparison:**
   - **Why:** Shows how normalization affects internal activation scales and distributions.
   - **Implementation:** Compute and visualize activation statistics across layers.

3. **Representational Similarity Analysis (CKA):**
   - **Why:** Quantifies similarity of internal representations between models.
   - **Implementation:** Use CKA to compare hidden states across models and layers.

4. **Probing Tasks:**
   - **Why:** Assesses what linguistic or semantic information is encoded in representations.
   - **Implementation:** Train simple classifiers on hidden states for diagnostic tasks.

5. **Weight Distribution and Norm Comparison:**
   - **Why:** Understands how normalization affects weight scaling and distribution.
   - **Implementation:** Analyze and visualize weight statistics and norms.

6. **Robustness and Stability Analysis:**
   - **Why:** Evaluates model resilience and generalization capabilities.
   - **Implementation:** Apply input perturbations and measure performance changes.

---

## **10. Integrating These Methods into Your Research**

### **a. Structuring Your Analysis Section**
Organize your analysis section to systematically present each method, its implementation, and the insights gained. For example:

1. **Gradient Analysis:**
   - Present your existing gradient visualization results.
   - Interpret the implications of higher gradient magnitudes in ablated models.

2. **Attention Head Analysis:**
   - Show visualizations of attention maps.
   - Discuss differences in attention patterns across models.

3. **Activation Distribution Comparison:**
   - Present statistics and visualizations of activations.
   - Analyze how normalization influences activation scales.

4. **Representational Similarity (CKA):**
   - Provide CKA scores comparing models.
   - Interpret similarities or divergences in internal representations.

5. **Probing Tasks:**
   - Describe the probing tasks and their outcomes.
   - Discuss how normalization affects the encoding of linguistic features.

6. **Weight Distribution Comparison:**
   - Present weight statistics and norm comparisons.
   - Analyze the impact of normalization on weight scaling.

7. **Robustness Analysis:**
   - Show performance metrics under perturbations.
   - Discuss the resilience of each model.

### **b. Ensuring Clarity and Conciseness**
Given the word limit, focus on the most impactful findings from each method. Use visual aids like tables, graphs, and heatmaps to succinctly present complex data.

### **c. Linking to Research Question**
Ensure that each method and its findings directly contribute to answering your research question about the impact of different layer normalization strategies. Highlight how each analysis provides unique insights into model behavior and performance.

---

## **11. Example Integration of Additional Methods**

Here’s an example of how you might integrate one additional method into your analysis section:

---

### **Attention Head Analysis**

To further understand the impact of different normalization strategies on the model's internal mechanisms, we conducted an attention head analysis. By visualizing the attention weights of each head across layers, we observed distinct patterns in how models with different normalization strategies focus on input tokens.

**Findings:**
- **Base Model:** Exhibits a balanced distribution of attention across heads, with some heads consistently focusing on syntactic elements like subjects and objects.
- **noNorm Model:** Attention maps are more diffuse, indicating less specialization among heads. This aligns with the higher gradient magnitudes observed, suggesting that without normalization, the model requires more adjustments to achieve optimal focus.
- **FFNonly and AttnOnly Models:** Show intermediate patterns, with specific heads demonstrating increased focus on certain token relationships, reflecting the selective application of normalization.

**Implications:**
These differences suggest that normalization strategies influence the specialization and efficiency of attention heads, potentially affecting how information is processed and integrated within the model.

---

## **12. Final Recommendations**

1. **Prioritize Impactful Methods:**
   - Choose methods that provide the most significant and relevant insights into your research question.
   - Focus on **Attention Head Analysis** and **Activation Distribution Comparison** as primary additional methods, given their direct relevance and manageability.

2. **Use Visual Aids Effectively:**
   - Employ heatmaps, line plots, and box plots to present complex data succinctly.
   - Ensure all visualizations are clearly labeled and referenced within your text.

3. **Maintain Focus and Coherence:**
   - Avoid overloading your essay with too many methods. Instead, select a few complementary techniques that together provide a comprehensive view.
   - Link each method’s findings back to your central research question to maintain narrative coherence.

4. **Document Methodology Clearly:**
   - Provide detailed explanations of how each analysis was conducted, ensuring reproducibility.
   - Include any scripts or tools used in an appendix if necessary, referencing them in your main text.

5. **Reflect on Findings Holistically:**
   - Synthesize insights from all analysis methods to present a unified understanding of how normalization strategies impact transformer models.
   - Discuss any conflicting findings and provide plausible explanations based on your observations.

---

## **Conclusion**

By incorporating additional analytical methods such as **Attention Head Analysis**, **Activation Distribution Comparison**, **Representational Similarity Analysis (CKA)**, **Probing Tasks**, **Weight Distribution Comparison**, and **Robustness Analysis**, you can deepen your understanding of how different layer normalization strategies influence transformer models. These methods complement your existing gradient visualization approach, providing a multifaceted evaluation that aligns with your research objectives while remaining manageable within the constraints of an IB Extended Essay.

Focusing on a select few impactful methods ensures that your analysis is both thorough and coherent, enabling you to present a compelling and insightful study. If you need further assistance with implementing any specific method or interpreting results, feel free to ask!